In [1]:
from functions import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import warnings
warnings.filterwarnings('ignore')

2023-12-14 12:17:40.799157: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# ANALYSIS

In [3]:
N = 1
profits = np.zeros(N)
for j in range(N):
	opp_stats = pd.read_csv('DATA_SPORT/Opponent Stats Per 100 Poss.csv')
	team_stats = pd.read_csv('DATA_SPORT/Team Stats Per 100 Poss.csv')
	game = pd.read_csv('DATA/GAME.csv')
	odds = pd.read_csv('DATA/ODDS.csv')
	odds = odds[odds['GAME_ID']!=0]
	odds = odds[odds['HOMEML'].notna()]
	useful_cols = ['season','abbreviation','fg_per_100_poss','x3p_per_100_poss','x2p_per_100_poss','ft_per_100_poss','orb_per_100_poss','drb_per_100_poss','ast_per_100_poss','stl_per_100_poss','blk_per_100_poss','tov_per_100_poss','pf_per_100_poss','pts_per_100_poss']
	team_stats = team_stats[useful_cols]
	for i in range(2,len(useful_cols)):
		useful_cols[i] = 'opp_' + useful_cols[i]
	opp_stats = opp_stats[useful_cols]
	stats = pd.merge(team_stats,opp_stats,on=['season','abbreviation'])
	useful_cols = ['GAME_ID','HOMETEAM','AWAYTEAM','WL_HOME','PLUS_MINUS_HOME','HOMEML']
	odds = odds[useful_cols]
	game_dates = game[['GAME_ID','GAME_DATE_HOME']].copy()
	game_dates['YEAR'] = pd.to_datetime(game_dates['GAME_DATE_HOME']).dt.year
	game_dates = game_dates.drop('GAME_DATE_HOME',axis=1)
	odds = pd.merge(odds,game_dates,on='GAME_ID',how='left')
	odds['YEAR-1'] = odds['YEAR'] - 1
	odds = pd.merge(odds, stats, left_on=['HOMETEAM', 'YEAR-1'], right_on=['abbreviation', 'season'], suffixes=('', '_Home'))
	odds = pd.merge(odds, stats, left_on=['AWAYTEAM', 'YEAR-1'], right_on=['abbreviation', 'season'], suffixes=('', '_Away'))
	drop_cols = ['YEAR-1','season','abbreviation','season_Away','abbreviation_Away','YEAR']
	odds = odds.drop(drop_cols,axis=1)
	data = odds.copy()
	state = random.randint(1,999)


	data = odds.copy()
	X = data.loc[:, 'fg_per_100_poss':]
	y = data['PLUS_MINUS_HOME']
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=state)

	scaler = StandardScaler()
	X_train_scaled = scaler.fit_transform(X_train)
	X_test_scaled = scaler.transform(X_test)

	model = Sequential([
		Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
		Dense(32, activation='relu'),
		Dense(1)  # Single output node for regression
	])

	model.compile(optimizer='adam', loss='mean_squared_error')
	model.fit(X_train_scaled, y_train, epochs=5, batch_size=4,verbose=1)
	loss = model.evaluate(X_test_scaled, y_test,verbose=0)
	print(f"\nTest Loss: {loss}")

	predictions = model.predict(X_test_scaled,verbose=0)[:,0]
	bin_truth = np.array(y_test) > 0
	bin_pred = predictions > 0
	count = 0 
	for i in range(len(bin_truth)):
		if bin_truth[i] == bin_pred[i]:
			count +=1
	print("Accuracy: {:<6.2f}%\n".format(count/len(predictions)*100))

	results = pd.DataFrame(y_test)
	results['GAME_ID'] = results.index.map(data['GAME_ID'])
	results['pred'] = predictions
	results = pd.merge(results, pd.read_csv('DATA/ODDS.csv'),on='GAME_ID')
	relevant_cols = ['GAME_ID','pred','HOMEML','AWAYML','WL_HOME','HOMESPREAD_ATCLOSE']
	results = results[relevant_cols]
	results['PRED_SPREAD'] = results['pred']*-1
	results['HWAGER']=0.0
	results['AWAGER']=0.0
	for i in range(len(results)):
		pred = results['PRED_SPREAD'].iloc[i]
		spread = results['HOMESPREAD_ATCLOSE'].iloc[i]
		awayml = results['AWAYML'].iloc[i]
		homeml = results['HOMEML'].iloc[i]
		# if spread <= 0:
		# 	if pred <= 0:
		# 		results.iloc[i,results.columns.get_loc('AWAGER')] = (abs(spread)+abs(pred))
		# 	elif pred >= -1*spread:
		# 		results.iloc[i,results.columns.get_loc('HWAGER')] = (abs(spread+pred))
		# elif spread > 0:
		# 	if pred > 0:
		# 		results.iloc[i,results.columns.get_loc('HWAGER')] = (abs(spread)+abs(pred))
		# 	elif pred < -1*spread:
		# 		results.iloc[i,results.columns.get_loc('AWAGER')] = (abs(spread+pred))
		if abs(spread-pred) >= 2:
			if (spread > 0 and pred > 0) or (spread < 0 and pred < 0):
				if awayml < 2:
					results.iloc[i,results.columns.get_loc('AWAGER')] = 1
			if (spread > 0 and pred < 0) or (spread < 0 and pred > 0):
				if homeml < 2:
					results.iloc[i,results.columns.get_loc('HWAGER')] = 1
	reg_results = results.copy()


	data = odds.copy()
	X = data.loc[:, 'fg_per_100_poss':]
	y = data['WL_HOME'].map({'W': 1, 'L': 0})
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=state)

	model = Sequential([
		Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
		Dense(32, activation='relu'),
		Dense(1, activation='sigmoid')  # Single output node for regression
	])

	model.compile(optimizer='adam', loss='binary_crossentropy')
	model.fit(X_train_scaled, y_train, epochs=5, batch_size=2,verbose=1)
	loss = model.evaluate(X_test_scaled, y_test,verbose=0)
	print(f"\nTest Loss: {loss}")

	predictions = model.predict(X_test_scaled,verbose=0)[:,0]
	bin_truth = np.array(y_test) > 0
	bin_pred = predictions > 0
	count = 0 
	for i in range(len(bin_truth)):
		if bin_truth[i] == bin_pred[i]:
			count +=1
	print("Accuracy: {:<6.2f}%\n".format(count/len(predictions)*100))

	results = pd.DataFrame(y_test)
	results['GAME_ID'] = results.index.map(data['GAME_ID'])
	results['pred'] = predictions
	results = pd.merge(results, pd.read_csv('DATA/ODDS.csv'),on='GAME_ID')
	results['WL_HOME'] = results['WL_HOME_y']
	relevant_cols = ['GAME_ID','pred','HOMEML','AWAYML','WL_HOME','HOMESPREAD_ATCLOSE']
	results = results[relevant_cols]
	results['PRED_SPREAD'] = results['pred']*-1
	results['HWAGER']=0.0
	results['AWAGER']=0.0
	for i in range(len(results)):
		homep = results['pred'].iloc[i]
		awayp = 1-homep
		homeml = results['HOMEML'].iloc[i]
		awayml = results['AWAYML'].iloc[i]

		if homep*homeml>1.1:
			results.iloc[i,results.columns.get_loc('HWAGER')] = 1
		if awayp*awayml>1.1:
			results.iloc[i,results.columns.get_loc('AWAGER')] = 1
	cla_results = results.copy()


	output = reg_results.copy().drop('pred',axis=1)
	print(len(output[output['HWAGER']==1]))
	print(len(output[output['AWAGER']==1]))
	output['PRED_ML'] = cla_results['pred']
	for i in range(len(output)):
		if output['HWAGER'].iloc[i] == 1:
			if output['PRED_ML'].iloc[i]*output['HOMEML'].iloc[i] < .98:
				output.iloc[i,output.columns.get_loc('HWAGER')] = 0
		if output['AWAGER'].iloc[i] == 1:
			if (1-output['PRED_ML'].iloc[i])*output['AWAYML'].iloc[i] < .98:
				output.iloc[i,output.columns.get_loc('AWAGER')] = 0
	print(len(output[output['HWAGER']==1]))
	print(len(output[output['AWAGER']==1]))
	output.to_csv('DATA/WAGERS.csv')
	profit = calc_results("DATA/WAGERS.csv","DATA/RESULTS.csv")
	profits[j] = profit
	print(j)

Epoch 1/5
3111/3111 [==============================] - 7s 2ms/step - loss: 174.6591
Epoch 2/5
3111/3111 [==============================] - 5s 2ms/step - loss: 171.0973
Epoch 3/5
3111/3111 [==============================] - 6s 2ms/step - loss: 168.7208
Epoch 4/5
3111/3111 [==============================] - 5s 2ms/step - loss: 166.3433
Epoch 5/5
3111/3111 [==============================] - 5s 2ms/step - loss: 164.2910

Test Loss: 171.330322265625
Accuracy: 62.84 %

Epoch 1/5
6221/6221 [==============================] - 12s 2ms/step - loss: 0.6594
Epoch 2/5
6221/6221 [==============================] - 11s 2ms/step - loss: 0.6457
Epoch 3/5
6221/6221 [==============================] - 11s 2ms/step - loss: 0.6406
Epoch 4/5
6221/6221 [==============================] - 11s 2ms/step - loss: 0.6329
Epoch 5/5
6221/6221 [==============================] - 11s 2ms/step - loss: 0.6273

Test Loss: 0.6350327134132385
Accuracy: 58.98 %

285
378
33
28
					$+2.87       on $     61.00:	  4.70%   profit.
	

In [17]:
profits

array([18.86314088,  5.45030041,  2.99741124,  4.5825057 ,  0.71145733,
        2.64777878,  4.83179334, 12.24315533,  5.72989093,  2.01016597,
        2.26817448,  7.57944266,  3.30734322, 11.69014367,  3.2402458 ])

# previous

In [21]:
data = odds.copy()
X = data.loc[:, 'fg_per_100_poss':]
y = data['PLUS_MINUS_HOME']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=state)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = Sequential([
	Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
	Dense(32, activation='relu'),
	Dense(1)  # Single output node for regression
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train_scaled, y_train, epochs=5, batch_size=4,verbose=1)
loss = model.evaluate(X_test_scaled, y_test,verbose=0)
print(f"Test Loss: {loss}\n")

predictions = model.predict(X_test_scaled,verbose=0)[:,0]
bin_truth = np.array(y_test) > 0
bin_pred = predictions > 0
count = 0 
for i in range(len(bin_truth)):
	if bin_truth[i] == bin_pred[i]:
		count +=1
print("Accuracy: {:<6.2f}%".format(count/len(predictions)*100))

results = pd.DataFrame(y_test)
results['GAME_ID'] = results.index.map(data['GAME_ID'])
results['pred'] = predictions
results = pd.merge(results, pd.read_csv('DATA/ODDS.csv'),on='GAME_ID')
relevant_cols = ['GAME_ID','pred','HOMEML','AWAYML','WL_HOME','HOMESPREAD_ATCLOSE']
results = results[relevant_cols]
results['PRED_SPREAD'] = results['pred']*-1
results['HWAGER']=0.0
results['AWAGER']=0.0
for i in range(len(results)):
	pred = results['PRED_SPREAD'].iloc[i]
	spread = results['HOMESPREAD_ATCLOSE'].iloc[i]
	awayml = results['AWAYML'].iloc[i]
	homeml = results['HOMEML'].iloc[i]
	# if spread <= 0:
	# 	if pred <= 0:
	# 		results.iloc[i,results.columns.get_loc('AWAGER')] = (abs(spread)+abs(pred))
	# 	elif pred >= -1*spread:
	# 		results.iloc[i,results.columns.get_loc('HWAGER')] = (abs(spread+pred))
	# elif spread > 0:
	# 	if pred > 0:
	# 		results.iloc[i,results.columns.get_loc('HWAGER')] = (abs(spread)+abs(pred))
	# 	elif pred < -1*spread:
	# 		results.iloc[i,results.columns.get_loc('AWAGER')] = (abs(spread+pred))
	if abs(spread-pred) >= 1:
		if (spread > 0 and pred > 0) or (spread < 0 and pred < 0):
			if awayml < 2:
				results.iloc[i,results.columns.get_loc('AWAGER')] = 1
		if (spread > 0 and pred < 0) or (spread < 0 and pred > 0):
			if homeml < 2:
				results.iloc[i,results.columns.get_loc('HWAGER')] = 1

reg_results = results.copy()

Epoch 1/5
3111/3111 [==============================] - 7s 2ms/step - loss: 174.7196
Epoch 2/5
3111/3111 [==============================] - 4s 1ms/step - loss: 170.4540
Epoch 3/5
3111/3111 [==============================] - 4s 1ms/step - loss: 168.0768
Epoch 4/5
3111/3111 [==============================] - 3s 1ms/step - loss: 165.9032
Epoch 5/5
3111/3111 [==============================] - 3s 1ms/step - loss: 163.6210
Test Loss: 171.3915557861328

Accuracy: 64.87 %
					$-35.56     on $    693.00:	 -5.13%   profit.
					453    of    693 games won:	 65.37% accuracy.


In [22]:
data = odds.copy()
X = data.loc[:, 'fg_per_100_poss':]
y = data['WL_HOME'].map({'W': 1, 'L': 0})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=state)

model = Sequential([
	Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
	Dense(32, activation='relu'),
	Dense(1, activation='sigmoid')  # Single output node for regression
])

model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(X_train_scaled, y_train, epochs=5, batch_size=2,verbose=1)
loss = model.evaluate(X_test_scaled, y_test,verbose=0)
print(f"Test Loss: {loss}\n")

predictions = model.predict(X_test_scaled,verbose=0)[:,0]
bin_truth = np.array(y_test) > 0
bin_pred = predictions > 0
count = 0 
for i in range(len(bin_truth)):
	if bin_truth[i] == bin_pred[i]:
		count +=1
print("Accuracy: {:<6.2f}%".format(count/len(predictions)*100))

results = pd.DataFrame(y_test)
results['GAME_ID'] = results.index.map(data['GAME_ID'])
results['pred'] = predictions
results = pd.merge(results, pd.read_csv('DATA/ODDS.csv'),on='GAME_ID')
results['WL_HOME'] = results['WL_HOME_y']
relevant_cols = ['GAME_ID','pred','HOMEML','AWAYML','WL_HOME','HOMESPREAD_ATCLOSE']
results = results[relevant_cols]
results['PRED_SPREAD'] = results['pred']*-1
results['HWAGER']=0.0
results['AWAGER']=0.0
for i in range(len(results)):
	homep = results['pred'].iloc[i]
	awayp = 1-homep
	homeml = results['HOMEML'].iloc[i]
	awayml = results['AWAYML'].iloc[i]

	if homep*homeml>1.1:
		results.iloc[i,results.columns.get_loc('HWAGER')] = 1
	if awayp*awayml>1.1:
		results.iloc[i,results.columns.get_loc('AWAGER')] = 1

cla_results = results.copy()
results.to_csv('DATA/WAGERS.csv')
profit = calc_results("DATA/WAGERS.csv","DATA/RESULTS.csv")

Epoch 1/5
6221/6221 [==============================] - 10s 1ms/step - loss: 0.6578
Epoch 2/5
6221/6221 [==============================] - 9s 1ms/step - loss: 0.6463
Epoch 3/5
6221/6221 [==============================] - 8s 1ms/step - loss: 0.6382
Epoch 4/5
6221/6221 [==============================] - 9s 1ms/step - loss: 0.6318
Epoch 5/5
6221/6221 [==============================] - 7s 1ms/step - loss: 0.6256
Test Loss: 0.6452802419662476

Accuracy: 59.37 %
					$-182.70    on $   2251.00:	 -8.12%   profit.
					672    of   2250 games won:	 29.87% accuracy.


In [6]:
output = reg_results.copy().drop('pred',axis=1)
print(len(output[output['HWAGER']==1]))
print(len(output[output['AWAGER']==1]))
output['PRED_ML'] = cla_results['pred']
for i in range(len(output)):
	if output['HWAGER'].iloc[i] == 1:
		if output['PRED_ML'].iloc[i]*output['HOMEML'].iloc[i] < 1:
			output.iloc[i,output.columns.get_loc('HWAGER')] = 0
	if output['AWAGER'].iloc[i] == 1:
		if (1-output['PRED_ML'].iloc[i])*output['AWAYML'].iloc[i] < 1:
			output.iloc[i,output.columns.get_loc('AWAGER')] = 0
print(len(output[output['HWAGER']==1]))
print(len(output[output['AWAGER']==1]))
output.to_csv('DATA/WAGERS.csv')
profit = calc_results("DATA/WAGERS.csv","DATA/RESULTS.csv")


381
500
48
98
					$+23.06      on $    146.00:	 15.79%   profit.
					103    of    146 games won:	 70.55% accuracy.


# New

In [79]:
opp_stats = pd.read_csv('DATA_SPORT/Opponent Stats Per 100 Poss.csv')
team_stats = pd.read_csv('DATA_SPORT/Team Stats Per 100 Poss.csv')
game = pd.read_csv('DATA/GAME.csv')
odds = pd.read_csv('DATA/ODDS.csv')

useful_cols = ['season','abbreviation','fg_per_100_poss','x3p_per_100_poss','x2p_per_100_poss','ft_per_100_poss','orb_per_100_poss','drb_per_100_poss','ast_per_100_poss','stl_per_100_poss','blk_per_100_poss','tov_per_100_poss','pf_per_100_poss','pts_per_100_poss']
team_stats = team_stats[useful_cols]
useful_cols = ['season', 'abbreviation', 'opp_fg_per_100_poss', 'opp_x3p_per_100_poss', 'opp_x2p_per_100_poss', 'opp_ft_per_100_poss', 'opp_orb_per_100_poss', 'opp_drb_per_100_poss', 'opp_ast_per_100_poss', 'opp_stl_per_100_poss', 'opp_blk_per_100_poss', 'opp_tov_per_100_poss', 'opp_pf_per_100_poss', 'opp_pts_per_100_poss']
opp_stats = opp_stats[useful_cols]
stats = pd.merge(team_stats,opp_stats,on=['season','abbreviation'])

useful_cols = ['GAME_ID','HOMETEAM','AWAYTEAM','WL_HOME','PLUS_MINUS_HOME','HOMEML','HOMESPREAD_ATOPEN']
odds = odds[useful_cols]
odds = odds[odds['GAME_ID']!=0]
odds = odds[odds['HOMEML'].notna()]
game_dates = game[['GAME_ID','GAME_DATE_HOME']].copy()
game_dates['YEAR'] = pd.to_datetime(game_dates['GAME_DATE_HOME']).dt.year
game_dates = game_dates.drop('GAME_DATE_HOME',axis=1)
odds = pd.merge(odds,game_dates,on='GAME_ID',how='left')
odds['YEAR-1'] = odds['YEAR'] - 1
odds = pd.merge(odds, stats, left_on=['HOMETEAM', 'YEAR-1'], right_on=['abbreviation', 'season'], suffixes=('', '_Home'))
odds = pd.merge(odds, stats, left_on=['AWAYTEAM', 'YEAR-1'], right_on=['abbreviation', 'season'], suffixes=('', '_Away'))
drop_cols = ['YEAR-1','season','abbreviation','season_Away','abbreviation_Away','YEAR']
odds = odds.drop(drop_cols,axis=1)

useful_cols = ['GAME_ID','TEAM_ABBREVIATION_HOME','TEAM_ABBREVIATION_AWAY','WL_HOME','PLUS_MINUS_HOME','GAME_DATE_HOME']
game = game[useful_cols]
game['YEAR'] = pd.to_datetime(game['GAME_DATE_HOME']).dt.year
game['YEAR-1'] = game['YEAR'] - 1
game = pd.merge(game, stats, left_on=['TEAM_ABBREVIATION_HOME', 'YEAR-1'], right_on=['abbreviation', 'season'], suffixes=('', '_Home'))
game = pd.merge(game, stats, left_on=['TEAM_ABBREVIATION_AWAY', 'YEAR-1'], right_on=['abbreviation', 'season'], suffixes=('', '_Away'))
drop_cols = ['YEAR-1','season','abbreviation','season_Away','abbreviation_Away','YEAR','GAME_DATE_HOME']
game = game[game['x3p_per_100_poss'].notna()].copy()
game = game.drop(drop_cols,axis=1)

X_train = game.loc[:, 'fg_per_100_poss':]
y_train_reg = game['PLUS_MINUS_HOME']
y_train_cla = game['WL_HOME'].map({'W':1,'L':0})
X_test = odds.loc[:, 'fg_per_100_poss':]
y_test_reg = odds['PLUS_MINUS_HOME']
y_test_cla = odds['WL_HOME'].map({'W':1,'L':0})

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

print('\nREGRESSION MODEL')

model = Sequential([
	Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
	Dense(32, activation='relu'),
	Dense(1)  # Single output node for regression
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train_reg, epochs=3, batch_size=4, verbose=1)
loss = model.evaluate(X_test, y_test_reg, verbose=0)
print(f"\nTest Loss: {loss}")

predictions = model.predict(X_test,verbose=0)[:,0]
bin_truth = np.array(y_test_reg) > 0
bin_pred = predictions > 0
count = 0 
for i in range(len(bin_truth)):
	if bin_truth[i] == bin_pred[i]:
		count +=1
print("Accuracy: {:<6.2f}%\n".format(count/len(predictions)*100))

results = pd.DataFrame(y_test_reg)
results['GAME_ID'] = odds['GAME_ID']
results['pred'] = predictions
results = pd.merge(results, pd.read_csv('DATA/ODDS.csv'), on='GAME_ID')
relevant_cols = ['GAME_ID','pred','HOMEML','AWAYML','WL_HOME','HOMESPREAD_ATCLOSE']
results = results[relevant_cols]
results['PRED_SPREAD'] = results['pred']*-1
results['HWAGER']=0.0
results['AWAGER']=0.0
for i in range(len(results)):
	pred = results['PRED_SPREAD'].iloc[i]
	spread = results['HOMESPREAD_ATCLOSE'].iloc[i]
	awayml = results['AWAYML'].iloc[i]
	homeml = results['HOMEML'].iloc[i]
	if abs(spread-pred) >= 2:
		if (spread > 0 and pred > 0) or (spread < 0 and pred < 0):
			if awayml < 2:
				results.iloc[i,results.columns.get_loc('AWAGER')] = 1
		if (spread > 0 and pred < 0) or (spread < 0 and pred > 0):
			if homeml < 2:
				results.iloc[i,results.columns.get_loc('HWAGER')] = 1
results.to_csv('DATA/WAGERS.csv')
calc_results("DATA/WAGERS.csv","DATA/RESULTS.csv")
reg_results = results.copy()

print('\nCLASSIFICATION MODEL')

model = Sequential([
	Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
	Dense(32, activation='relu'),
	Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(X_train, y_train_cla, epochs=3, batch_size=4,verbose=1)
loss = model.evaluate(X_test, y_test_cla,verbose=0)
print(f"\nTest Loss: {loss}")

predictions = model.predict(X_test,verbose=0)[:,0]
bin_truth = np.array(y_test_cla) > 0
bin_pred = predictions > 0
count = 0 
for i in range(len(bin_truth)):
	if bin_truth[i] == bin_pred[i]:
		count +=1
print("Accuracy: {:<6.2f}%\n".format(count/len(predictions)*100))

results = pd.DataFrame(y_test_cla)
results['GAME_ID'] = odds['GAME_ID']
results['pred'] = predictions
results = pd.merge(results, pd.read_csv('DATA/ODDS.csv'),on='GAME_ID')
results['WL_HOME'] = results['WL_HOME_y']
relevant_cols = ['GAME_ID','pred','HOMEML','AWAYML','WL_HOME','HOMESPREAD_ATCLOSE']
results = results[relevant_cols]
results['PRED_SPREAD'] = results['pred']*-1
results['HWAGER']=0.0
results['AWAGER']=0.0
for i in range(len(results)):
	homep = results['pred'].iloc[i]
	awayp = 1-homep
	homeml = results['HOMEML'].iloc[i]
	awayml = results['AWAYML'].iloc[i]

	if homep*homeml>1.1:
		results.iloc[i,results.columns.get_loc('HWAGER')] = 1
	if awayp*awayml>1.1:
		results.iloc[i,results.columns.get_loc('AWAGER')] = 1
results.to_csv('DATA/WAGERS.csv')
calc_results("DATA/WAGERS.csv","DATA/RESULTS.csv")
cla_results = results.copy()

print("\nCOMBINED MODEL")

output = reg_results.copy().drop('pred',axis=1)
output['PRED_ML'] = cla_results['pred']
for i in range(len(output)):
	if output['HWAGER'].iloc[i] == 1:
		if output['PRED_ML'].iloc[i]*output['HOMEML'].iloc[i] < .98:
			output.iloc[i,output.columns.get_loc('HWAGER')] = 0
	if output['AWAGER'].iloc[i] == 1:
		if (1-output['PRED_ML'].iloc[i])*output['AWAYML'].iloc[i] < .98:
			output.iloc[i,output.columns.get_loc('AWAGER')] = 0
output.to_csv('DATA/WAGERS.csv')
profit = calc_results("DATA/WAGERS.csv","DATA/RESULTS.csv")


REGRESSION MODEL
Epoch 1/3
9161/9161 [==============================] - 19s 2ms/step - loss: 160.1451
Epoch 2/3
9161/9161 [==============================] - 17s 2ms/step - loss: 157.8743
Epoch 3/3
9161/9161 [==============================] - 19s 2ms/step - loss: 157.0991

Test Loss: 172.66351318359375
Accuracy: 62.98 %

					$-123.35    on $   2534.00:	 -4.87%   profit.
					1704   of   2534 games won:	 67.25% accuracy.

CLASSIFICATION MODEL
Epoch 1/3
9161/9161 [==============================] - 20s 2ms/step - loss: 0.6427
Epoch 2/3
9161/9161 [==============================] - 19s 2ms/step - loss: 0.6356
Epoch 3/3
9161/9161 [==============================] - 20s 2ms/step - loss: 0.6329

Test Loss: 0.6632084846496582
Accuracy: 59.05 %

					$-1.46      on $  10982.00:	 -0.01%   profit.
					4345   of  10981 games won:	 39.57% accuracy.

COMBINED MODEL
					$-16.00     on $    363.00:	 -4.41%   profit.
					222    of    363 games won:	 61.16% accuracy.


In [78]:
output

,GAME_ID,HOMEML,AWAYML,WL_HOME,HOMESPREAD_ATCLOSE,PRED_SPREAD,HWAGER,AWAGER,PRED_ML
0,20700001,1.071429,10.000000,W,-13.0,-21.690615,0.0,0.0,0.879853
1,20700243,1.066667,10.500000,W,-13.5,-21.690615,0.0,0.0,0.879853
2,20700252,1.235294,4.550000,L,-8.5,-16.894398,0.0,0.0,0.778078
3,20700114,1.142857,6.500000,W,-11.0,-12.452097,0.0,0.0,0.742249
4,20700352,1.194175,5.150000,L,-9.0,-12.452097,0.0,0.0,0.742249
...,...,...,...,...,...,...,...,...,...
15547,22000694,1.476190,2.800000,W,-5.5,-3.237256,0.0,0.0,0.612926
15548,22000856,1.526316,2.700000,W,-4.5,-3.237256,0.0,0.0,0.612926
15549,22000272,1.769231,2.100000,W,-2.0,1.151206,1.0,0.0,0.554483
15550,22100176,1.540541,2.650000,W,-4.5,1.151206,0.0,0.0,0.554483
